In [12]:
from bs4 import BeautifulSoup
import requests
import urllib.request as req
import pandas as pd
import folium
import webbrowser

# 1단계 : 서울 문화공간 위치정보(주소) 획득
key = '4a6353746b796a68333079486b464e' # 7549474863796a68383358725a7975
url1 = 'http://openAPI.seoul.go.kr:8088/' + key + '/xml/SearchCulturalFacilitiesAddressService/1/1000'
save_attrfile_path = 'C:/Users/Performance/Desktop/Fist/src/ch15_attr_xmlfile.xml'
req.urlretrieve(url1, save_attrfile_path)

attr_xml = open(save_attrfile_path, 'r', encoding='utf-8').read() 
soup = BeautifulSoup(attr_xml, 'xml')

attr_list = []
for row in soup.select('row'):
    fac_code = row.select_one('FAC_CODE').text
    subjcode = row.select_one('SUBJCODE').text
    codename = row.select_one('CODENAME').text
    fac_name = row.select_one('FAC_NAME').text
#     main_img = row.select_one('MAIN_IMG') ######AttributeError: 'NoneType' object has no attribute 'text'##########
    addr = row.select_one('ADDR').text.replace(' )','')
#     attr_list.append( {'fac_code' : fac_code, 'subj_code' : subjcode, 'codename' : codename, 'fac_name' : fac_name, 'main_img' : main_img, 'addr' : addr} )
    attr_list.append( {'fac_code' : fac_code, 'subj_code' : subjcode, 'codename' : codename, 'fac_name' : fac_name, 'addr' : addr} )
df_attr = pd.DataFrame(attr_list)



# 2단계 : 위치정보(주소)를 좌표로 변환
url2 = 'http://api.vworld.kr/req/address?service=address&request=getCoord&version=2.0&crs=epsg:4326&format=json&key=EA773332-297D-38CA-9A54-76F66D21A455&type=parcel'
addr_list = []
for idx, addr in enumerate(df_attr['addr']):
    try:
        resp = requests.get(url2 + '&address=' + addr)
        addr_json = resp.json()
        if addr_json['response']['status'] == 'OK':
            dic = {}
            dic['level2'] = addr_json['response']['refined']['structure']['level2']
            dic['x'] = addr_json['response']['result']['point']['x']
            dic['y'] = addr_json['response']['result']['point']['y']
            addr_list.append(dic)
        elif addr_json['response']['status'] == 'NOT_FOUND':
            url2 = 'http://api.vworld.kr/req/address?service=address&request=getCoord&version=2.0&crs=epsg:4326&format=json&key=EA773332-297D-38CA-9A54-76F66D21A455&type=road'
            resp2 = requests.get(url2 + '&address=' + addr)
            addr_json = resp2.json()
            if addr_json['response']['status'] == 'OK':
                dic = {}
                dic['level2'] = addr_json['response']['refined']['structure']['level2']
                dic['x'] = addr_json['response']['result']['point']['x']
                dic['y'] = addr_json['response']['result']['point']['y']
                addr_list.append(dic)
            else:
                dic = {}
                dic['level2'] = None
                dic['x'] = None
                dic['y'] = None
                addr_list.append(dic)                
        else:
            dic = {}
            dic['level2'] = None
            dic['x'] = None
            dic['y'] = None
            addr_list.append(dic)
    except Exception:
        dic = {}
        dic['level2'] = None
        dic['x'] = None
        dic['y'] = None
        addr_list.append(dic)
df_addr = pd.DataFrame(addr_list)

if df_attr.iloc[:,0].size == df_addr.iloc[:,0].size:
    df_combine = pd.concat([df_attr, df_addr], axis = 1)
else:
    print('두 데이터프레임의 행 수가 일치하지 않습니다.')

# 기능 정의 : fn1 - 행정구역별 문화시설 조회
def fn1_by_district():

    gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
          '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
          '종로구', '중구', '중랑구' ]
    dist_list = []
    while True:
        dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함) : ')
        if dist_input == '':
            break
        if (dist_input in gu) == True:
            dist_list.append(dist_input)
        else:
            print('입력값을 다시 확인해 주세요')
            
    df_list = []
    for dist in dist_list:
        df_list.append(df_combine[df_combine['level2']==dist])    
    df_result1 = pd.concat(df_list)

    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result1['y'].astype('float').mean()
    long_mean = df_result1['x'].astype('float').mean()
    m = folium.Map([lat_mean, long_mean], zoom_start = 14)

    result1_by_subj = []
    for idx, i in enumerate(df_result1['subj_code'].unique()): # enumerate를 생각해낸게 해결법에 가까워진듯.
        result1_by_subj.append(df_result1[df_result1['subj_code'] == i])    
        for j in result1_by_subj[idx].index:
            tooltip = df_result1.loc[j, 'codename'] + ' - ' + df_result1.loc[j, 'fac_name'] + ' - ' + df_result1.loc[j, 'addr']
            lat = df_result1.loc[j, 'y']
            long = df_result1.loc[j, 'x']   # result1 대신 result1_by_subj가 들어가야 하지 않을까 ?
            folium.CircleMarker(
                location = [lat, long],
                radius = 5,
                color = hex_color[idx],
                tooltip = tooltip
            ).add_to(m)

    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')

# 기능 정의 : fn2 - 문화시설 분류별 조회
def fn2_by_type():

    attr_type_list = []
    while True:
        type_input = input('[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요 : ')
        if type_input == '':
            break
        attr_type_list.append(type_input)
    
    type_matched_list = []
    for attr_type in attr_type_list:
        type_matched_list.append(df_combine[df_combine['codename'] == attr_type])
    df_result2 = pd.concat(type_matched_list)
    df_result2.dropna(axis = 0, how = 'any', inplace = True)
    
    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result2['y'].astype('float').mean()
    long_mean = df_result2['x'].astype('float').mean()
    m = folium.Map([lat_mean, long_mean], zoom_start = 14)
    
    for idx, k in enumerate(attr_type_list):
        for l in df_result2[df_result2['codename'] == k].index:
            tooltip = df_result2.loc[l,'codename'] + ' - ' + df_result2.loc[l, 'fac_name'] + ' - '+ df_result2.loc[l, 'addr']
            lat = df_result2.loc[l, 'y']
            long = df_result2.loc[l, 'x']
            folium.CircleMarker(
                location = [lat, long],
                radius = 5,
                color = hex_color[idx],
                tooltip = tooltip
            ).add_to(m)
    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')

# 기능 정의 : fn3 - 다중조건 조회
def fn3_by_multi():
    import random
    gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
      '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
      '종로구', '중구', '중랑구' ]
    dist_list = []
    while True :
        dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요 : ')
        if (dist_input in gu) == True:
            dist_list.append(dist_input)
            break
        else:
            print('정확한 행정구역 명을 입력해 주세요') 
        
    attr_type = input('[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요')
    df_result3 = df_combine[(df_combine['level2'] == dist_input) & (df_combine['codename'] == attr_type)]

    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result3['y'].astype('float').mean()
    long_mean = df_result3['x'].astype('float').mean()
    
    m = folium.Map([lat_mean, long_mean], zoom_start=12)
    for n in df_result3.index:
        lat = df_result3.loc[n, 'y']
        long = df_result3.loc[n, 'x']
        tooltip = df_result3.loc[n, 'codename'] + ' - ' + df_result3.loc[n, 'fac_name'] + ' - ' + df_result3.loc[n , 'addr']
        folium.CircleMarker(
            location = [lat, long],
            radius = 5,
            color = hex_color[random.randrange(len(hex_color))],
            tooltip = tooltip,
        ).add_to(m)
    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')
    
def main():
    while True:
        print('1: 행정구역 내 관광지 | 2: 장르별 관광지 | 3: 행정구역 내 장르별 관광지 | 9: 종료')
        fn = int(input('선택 : '))
        if fn == 1:
            fn1_by_district()
        elif fn == 2:
            fn2_by_type()
        elif fn == 3:
            fn3_by_multi()
        elif fn == 9:
            break

In [13]:
if __name__ == '__main__':
    main()

1: 행정구역 내 관광지 | 2: 장르별 관광지 | 3: 행정구역 내 장르별 관광지 | 9: 종료
선택 : 9


In [6]:
gu = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
      '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
      '종로구', '중구', '중랑구']
dist = input('희망하는 지역의 행정구역(구)를 입력해주세요')
dist in gu

희망하는 지역의 행정구역(구)를 입력해주세요성동구


True

In [18]:
dist_list

['강북구']

In [5]:
class RaisePass(Exception):
    pass

gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
  '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
  '종로구', '중구', '중랑구' ]
dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요')
try:
    if (dist_input in gu) == True:
        raise RaisePass
except RaisePass:
    print('입력값을 다시 확인해주세요')

희망하는 지역의 행정구역(구)를 입력해주세요saas


In [66]:
def fn3_by_multi():
    import random
    gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
      '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
      '종로구', '중구', '중랑구' ]
    dist_list = []
    while True :
        dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요')
        if (dist_input in gu) == True:
            dist_list.append(dist_input)
            break
        else:
            print('정확한 행정구역 명을 입력해 주세요') 
        
    attr_type = input('[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요')
    df_result3 = df_combine[(df_combine['level2'] == dist_input) & (df_combine['codename'] == attr_type)]

    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result3['y'].astype('float').mean()
    long_mean = df_result3['x'].astype('float').mean()
    
    m = folium.Map([lat_mean, long_mean], zoom_start=12)
    for n in df_result3.index:
        lat = df_result3.loc[n, 'y']
        long = df_result3.loc[n, 'x']
        tooltip = df_result3.loc[n, 'codename'] + ' - ' + df_result3.loc[n, 'fac_name'] + ' - ' + df_result3.loc[n , 'addr']
        folium.CircleMarker(
            location = [lat, long],
            radius = 5,
            color = hex_color[random.randrange(len(hex_color))],
            tooltip = tooltip,
        ).add_to(m)
    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')

In [67]:
fn3_by_multi()

희망하는 지역의 행정구역(구)를 입력해주세요종로구
[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요미술관
